# Tópicos em IA — Modelos de Linguagem
## Aula: Word Embeddings Clássicos — **Word2Vec, GloVe e FastText**

**Pré-requisitos:** Python básico, cálculo, estatística introdutória.

### Objetivos
- Entender, em detalhe, os modelos **Word2Vec (CBOW e Skip-gram)**, **GloVe** e **FastText**.
- Derivar as **funções objetivo** e os **gradientes** centrais.
- Implementar mini-versões dos algoritmos para compreender a mecânica de treinamento.
- Comparar qualitativamente os modelos (semântica, morfologia, OOV).
- Exercitar avaliação intrínseca (similaridade/analogia) e discutir vieses.

> Referências base: notas do **CS224n** (Manning) sobre Word Vectors e GloVe; ver bibliografia ao final.

## Sumário
1) Motivação: do *one-hot* a vetores densos  
2) **Word2Vec**: softmax, *negative sampling*, CBOW vs. Skip-gram; **derivações**  
3) **GloVe**: matriz de coocorrência, objetivo ponderado, **derivações**  
4) **FastText**: n-gramas de caracteres, OOV, morfologia  
5) Avaliação (intrínseca/extrínseca), analogias, TSNE  
6) Atividades guiadas, ética e vieses, ganchos para a próxima aula

In [18]:
# Instalação das dependências localmente
# Se estiver rodando localmente, descomente a linha abaixo para instalar as dependências
# ! pip install -r requirements.txt

In [19]:
# Se rodando no Google Colab, descomente a linha abaixo para montar o Google Drive
# from google.colab import drive
# drive.mount('/content/drive/', force_remount=True)

In [20]:
# Instalação das dependências no Google Colab
# Mude CAMINHO_PARA_REPO para o caminho correto do seu repositório no seu Google Drive
# ! pip install -r /content/drive/MyDrive/UFMS/Aulas/2025-2/TOPIA/repo/TopicosIA-2025-02/requirements.txt

## 1) Motivação: do one-hot para embeddings

**Problema do one-hot:** vetores são ortogonais; não há noção de similaridade.

Sejam duas palavras `hotel` e `motel`:
$$
\mathbf{e}_{\text{hotel}},\ \mathbf{e}_{\text{motel}} \in \mathbb{R}^{|V|},\quad
\mathbf{e}_{\text{hotel}}^\top \mathbf{e}_{\text{motel}} = 0
$$

**Ideia:** aprender **vetores densos** $\mathbf{w}\in\mathbb{R}^d$ (com $d\ll |V|$) onde a proximidade (p.ex. cosseno) reflete semântica:
$$
\cos(\mathbf{w}_a,\mathbf{w}_b)=\frac{\mathbf{w}_a^\top \mathbf{w}_b}{\|\mathbf{w}_a\|\,\|\mathbf{w}_b\|}
$$

**Hipótese distribucional:** “Conhecerás a palavra pela companhia que ela mantém.” (Firth, 1957).

In [21]:
import numpy as np

V = 6
idx = {"hotel": 1, "motel": 4}
e_hotel = np.eye(V)[idx["hotel"]]
e_motel = np.eye(V)[idx["motel"]]

cos = e_hotel @ e_motel / (np.linalg.norm(e_hotel)*np.linalg.norm(e_motel) + 1e-12)
print("Cosseno(one-hot hotel, motel) =", cos)  # 0.0

rng = np.random.default_rng(0)
w_hotel = rng.normal(size=50)
w_motel = w_hotel + 0.1*rng.normal(size=50)  
cos_dense = (w_hotel @ w_motel)/(np.linalg.norm(w_hotel)*np.linalg.norm(w_motel))
print("Cosseno(embeddings densos) ≈", float(cos_dense))

Cosseno(one-hot hotel, motel) = 0.0
Cosseno(embeddings densos) ≈ 0.9939728426989571


## 2) Word2Vec (Mikolov et al., 2013)

Dadas sequências $(w_1,\dots,w_T)$, com janela de tamanho $m$:

- **Skip-gram:** prediz contexto $w_{t+j}$ a partir da palavra central $w_t$.
- **CBOW:** prediz a central $w_t$ a partir do **bag** dos contextos $\{w_{t\pm j}\}$.

<br/>
<img src="https://raw.githubusercontent.com/bmnogueira-ufms/TopicosIA-2025-02/main/images/skip-gram-cbow.png" width="55%">

### Probabilidade via softmax (modelo "naïve softmax")
Para central $c$ e contexto $o$, com **dois** vetores por palavra (centro $\mathbf{v}_w$, contexto $\mathbf{u}_w$):
$$
P(o\mid c)=\frac{\exp(\mathbf{u}_o^\top \mathbf{v}_c)}{\sum_{w\in V}\exp(\mathbf{u}_w^\top \mathbf{v}_c)}
$$


**Passo a passo da intuição:**

1. **Produto interno ($\mathbf{u}_o^\top \mathbf{v}_c$):**  
   Mede a *similaridade* entre os vetores da palavra central $c$ e da candidata $o$.  
   - Alto $\rightarrow$ palavras aparecem juntas com frequência.  
   - Baixo $\rightarrow$ palavras raramente aparecem juntas.

2. **Exponenciação ($\exp(\cdot)$):**  
   - Garante valores positivos.  
   - Amplifica diferenças: pequenas vantagens viram grandes diferenças de peso.

3. **Normalização (softmax):**  
   - Divide pelo somatório de todos os candidatos do vocabulário.  
   - Cria uma **distribuição de probabilidade** sobre todas as palavras.  

---

**Intuição global:**  
O modelo responde à pergunta:  
> *“Dada a palavra central $c$, qual a probabilidade de $o$ estar em seu contexto?”*  

Se a compatibilidade (produto interno) entre $c$ e $o$ é alta, a probabilidade cresce.  
Se é baixa, a probabilidade cai.  

---

**Objetivo (Skip-gram):**
$$
J(\theta)=-\frac{1}{T}\sum_{t=1}^T \sum_{-m\le j\le m,\ j\ne 0}\ \log P(w_{t+j}\mid w_t)
$$

O softmax requer normalização sobre $V$ → caro para vocabulários grandes.  
Duas soluções: **Negative Sampling** e **Hierarchical Softmax**.

### Derivações (naïve softmax, Skip-gram)

Para um par $(c,o)$ e predição $\hat{\mathbf{y}}=\text{softmax}(U\mathbf{v}_c)$, com rótulo one-hot $\mathbf{y}$ s.t. $y_o=1$:

- $\mathbf{v}_c \in \mathbb{R}^d$ → vetor da palavra **central** $c$ (embedding de dimensão $d$).  
- $U \in \mathbb{R}^{|V|\times d}$ → matriz que contém os **vetores de contexto** (um para cada palavra do vocabulário).  
  - Cada linha $\mathbf{u}_w^\top$ de $U$ é o vetor associado à palavra $w$ quando ela atua como **contexto**.

  - **Produto interno com todos os contextos**  
    - Para cada palavra $w$ do vocabulário, calcula $\mathbf{u}_w^\top \mathbf{v}_c$.  
    - Esse valor é um **escore de compatibilidade** entre $c$ (central) e $w$ (possível contexto).  

  - **Resultado vetorial**  
    - O produto $U \mathbf{v}_c$ produz um vetor de dimensão $|V|$.  
    - Cada posição corresponde ao “quanto o modelo acredita” que aquela palavra pode ser contexto de $c$.  

  - **Softmax transforma em probabilidade**  
    - Em seguida, aplicamos:
     $$
     \hat{\mathbf{y}} = \text{softmax}(U \mathbf{v}_c)
     $$
    - Assim, cada escore vira uma **probabilidade normalizada** de a palavra ser o contexto verdadeiro.

**Perda por par:**
$$
J_{\text{pair}} = -\log P(o\mid c)
$$

- $c$ = palavra **central**  
- $o$ = palavra **de contexto**  
- $P(o \mid c)$ = probabilidade atribuída pelo modelo de observar $o$ no contexto de $c$  

---

#### Como interpretar

1. **Maximizar probabilidade correta**  
   - Se o modelo acha que $o$ é muito provável dado $c$, então $P(o\mid c) \approx 1$ → perda próxima de 0.  
   - Se o modelo acha que $o$ é improvável, $P(o\mid c) \ll 1$ → perda grande.

2. **Por que o log?**  
   - Evita lidar com produtos de probabilidades muito pequenas.  
   - Cada par $(c,o)$ contribui de forma aditiva ao custo total.  

3. **Por que o sinal de menos?**  
   - Como $\log P \leq 0$, o “menos” transforma em valor **positivo**.  
   - Isso equivale a **minimizar a perda** em vez de maximizar a probabilidade.

---

#### Intuição global
A função está dizendo:  

*“Quero que a probabilidade atribuída ao par correto $(c,o)$ seja a maior possível.”*

- Se $P(o\mid c)$ é alto → perda pequena.  
- Se $P(o\mid c)$ é baixo → perda grande.  

---

**Gradientes:**
$$
\frac{\partial J_{\text{pair}}}{\partial \mathbf{v}_c} = U^\top(\hat{\mathbf{y}}-\mathbf{y}),
\qquad
\frac{\partial J_{\text{pair}}}{\partial \mathbf{u}_w} = (\hat{y}_w - y_w)\,\mathbf{v}_c,\ \forall w\in V
$$

Intuição: $(\hat{\mathbf{y}}-\mathbf{y})$ é o **erro**; atualizamos $\mathbf{v}_c$ e as $\mathbf{u}_w$ proporcionais a esse erro.  
Complexidade ainda é $O(|V|)$ por atualização → motivação para **Negative Sampling**.

### Negative Sampling (Skip-gram)

**Ideia:** treinar classificadores binários que **distingam** um par real $(c,o)$ de $k$ pares negativos $(c,w_i)$ amostrados de uma distribuição de ruído $P_n$.

**Objetivo por par $(c,o)$ e amostras $\{w_i\}_{i=1}^k$:**
$$
J_{\text{NS}} = -\log \sigma(\mathbf{u}_o^\top \mathbf{v}_c)\;-\; \sum_{i=1}^k \log \sigma\!\big(-\mathbf{u}_{w_i}^\top \mathbf{v}_c\big),
$$
onde $\sigma(x)=\tfrac{1}{1+e^{-x}}$.

**Gradientes:**
$$
\frac{\partial J_{\text{NS}}}{\partial \mathbf{v}_c} = \big(\sigma(\mathbf{u}_o^\top \mathbf{v}_c)-1\big)\mathbf{u}_o
\;+\; \sum_{i=1}^k \sigma(\mathbf{u}_{w_i}^\top \mathbf{v}_c)\,\mathbf{u}_{w_i}
$$

$$
\frac{\partial J_{\text{NS}}}{\partial \mathbf{u}_o} = \big(\sigma(\mathbf{u}_o^\top \mathbf{v}_c)-1\big)\mathbf{v}_c,
\qquad
\frac{\partial J_{\text{NS}}}{\partial \mathbf{u}_{w_i}} = \sigma(\mathbf{u}_{w_i}^\top \mathbf{v}_c)\,\mathbf{v}_c
$$

**Prática:** amostrar negativos com $P_n(w)\propto U(w)^{3/4}$.

In [22]:
corpus = [
    "o brasil gosta de futebol",
    "o rio tem praia bonita",
    "sao paulo tem universidade e hospital",
    "a amazonia tem selva e rios",
    "o brasil produz cafe",
    "o rio de janeiro tem futebol e praia",
    "sao paulo tem hospital famoso",
    "a universidade pesquisa inteligencia artificial",
    "cafe do brasil é famoso",
    "a selva da amazonia é densa",
    "o hospital do rio é universitario",

    "o cruzeiro é o melhor time do brasil",
    "o time de futebol joga no estadio",
    "a torcida canta no estadio",
    "o jogador marcou um gol",
    "o tecnico escala o time titular",
    "o campeonato brasileiro tem rodada no domingo",
    "o arbitro apita o jogo de futebol",
    "o atacante fez dois gols no classico",
    "o goleiro defendeu o penalti",
    "o time treinou no campo de grama",
    "o maracana fica no rio de janeiro",
    "o morumbi fica em sao paulo",
    "o estadio recebe finais importantes",
    "o time carioca venceu o jogo",
    "o time paulista empatou fora de casa",
    "a base revela novos jogadores",
    "o lateral cruza a bola para a area",
    "o meio campista organiza o ataque",
    "o zagueiro corta o cruzamento",
    "o treinador analisa estatisticas do jogo",

    "a praia de copacabana tem areia branca",
    "ipanema tem mar e sol",
    "o turista caminha no calcadao de copacabana",
    "o surfista pega onda na praia",
    "banhistas lotam a praia no verao",
    "o quiosque vende agua de coco",
    "o rio tem orla movimentada",
    "a brisa do mar refresca a tarde",
    "a praia tem guarda sol e cadeira",
    "o por do sol em ipanema é bonito",
    "o passeio de bicicleta na orla é popular",

    "a universidade tem campus grande",
    "o campus tem biblioteca e laboratorio",
    "o professor orienta estudante de mestrado",
    "a pesquisa em dados utiliza python",
    "o laboratorio de ia treina redes neurais",
    "a pos graduacao publica artigos cientificos",
    "o grupo de pesquisa organiza seminario semanal",
    "a universidade federal tem curso de computacao",
    "o aluno estuda no laboratorio de informatica",
    "a disciplina ensina machine learning",
    "o dataset tem noticias do brasil",
    "a avaliacao usa acuracia e f1",
    "o modelo aprende representacoes distribuídas",
    "o repositorio guarda codigo e dados",
    "o professor apresenta resultados no congresso",
    "o curso tem prova e projeto final",
    "a biblioteca empresta livros e revistas",
    "a universidade tem hospital universitario",
    "a pesquisa em linguagem natural usa word2vec",
    "o laboratorio de visao computacional anota imagens",
    "a turma pratica classificacao de textos",
    "o servidor treina o modelo com gpu",
    "a extensao conecta universidade e comunidade",
    "o estagio aproxima alunos do mercado",

    "o hospital universitario atende pacientes",
    "o medico realiza cirurgia no centro cirurgico",
    "a enfermeira aplica vacina na sala de atendimento",
    "o paciente marcou consulta com o especialista",
    "a emergencia recebe ambulancia a noite",
    "o laboratorio realiza exame de sangue",
    "o hospital tem leitos e uti",
    "a saude publica contrata medicos",
    "o medico receita medicamento para o paciente",
    "o hospital de sao paulo e referencia",
    "o pronto socorro fica lotado no feriado",
    "a vacina previne doenca infecciosa",
    "a farmacia do hospital entrega remedio",
    "a triagem organiza a fila de atendimento",
    "o prontuario registra historico do paciente",

    "a amazonia abriga grande biodiversidade",
    "o rio amazonas corta a floresta",
    "o rio negro encontra o rio solimoes",
    "a floresta tem arvores altas e lianas",
    "a chuva e intensa na amazonia",
    "comunidades indigenas vivem na regiao",
    "o pesquisador estuda fauna e flora",
    "o boto cor de rosa nada no rio",
    "o peixe e alimento importante na regiao",
    "o parque nacional protege a mata",
    "o desmatamento ameaca a floresta",
    "o ibama fiscaliza a regiao amazonica",
    "o clima e umido e quente na floresta",
    "barcos transportam pessoas pelos rios",
    "manaus tem porto no rio negro",
    "belem fica proxima da foz do amazonas",

    "o cafe de minas gerais tem qualidade",
    "a fazenda colhe graos maduros de cafe",
    "a torra define o sabor do cafe",
    "o barista prepara espresso na cafeteria",
    "a xicara de cafe acompanha o pao de queijo",
    "o brasil exporta cafe para a europa",
    "o cafe especial recebe pontuacao alta",
    "sao paulo tem muitas cafeterias",
    "o produtor investe em irrigacao por gotejamento",
    "a cooperativa vende graos torrados",
    "o cafe arabica cresce em altitude",
    "o cafe robusta cresce no espirito santo",

    "brasilia e a capital do brasil",
    "belo horizonte tem feira de artesanato",
    "curitiba tem parque bonito e organizado",
    "salvador tem carnaval famoso",
    "fortaleza tem praia do futuro movimentada",
    "recife tem o porto digital de tecnologia",
    "porto alegre tem inverno frio",
    "florianopolis tem muitas praias e trilhas",
    "natal tem duna e passeio de buggy",
    "gramado recebe muitos turistas no inverno",

    "o estadio maracana recebe finais e classicos de futebol",
    "o morumbi recebe shows e jogos de futebol",
    "o campus da universidade tem hospital universitario",
    "o hospital universitario participa de pesquisa clinica",
    "a universidade organiza torneio de futebol entre cursos",
    "a praia de copacabana recebe turistas e atletas",
    "o laboratorio de dados analisa estatisticas do campeonato",
    "o time treina na academia do clube",
    "a cafeteria do campus serve cafe especial",
    "o congresso de ia ocorre em sao paulo",
    "o museu do futebol fica no estadio do pacaembu",
    "o centro de inovacao conecta startups e universidade",
]

In [23]:
# Mini Skip-gram com Negative Sampling (NumPy)
import numpy as np
from collections import Counter, defaultdict
rng = np.random.default_rng(42)

# --- 1) Corpus e preparação ---
sentences = [s.split() for s in corpus]
window = 2
tokens = [w for s in sentences for w in s]
vocab = sorted(set(tokens))
word2id = {w:i for i,w in enumerate(vocab)}
id2word = {i:w for w,i in word2id.items()}
ids = [word2id[w] for w in tokens]

# Pares (central, contexto)
pairs = []
for s in sentences:
    ids_s = [word2id[w] for w in s]
    for t, c in enumerate(ids_s):
        L = max(0, t-window); R = min(len(ids_s), t+window+1)
        for j in range(L, R):
            if j==t: continue
            pairs.append((c, ids_s[j]))

V = len(vocab)
print("Vocabulário:", V, "Pares:", len(pairs))

# Frequências para negativos (unigrama^0.75)
freq = Counter(ids)
unigram = np.array([freq[i] for i in range(V)], dtype=np.float64)
noise_dist = unigram**0.75
noise_dist /= noise_dist.sum()

def sample_negatives(k):
    return rng.choice(V, size=k, replace=True, p=noise_dist)

# --- 2) Parâmetros ---
d = 50
lr = 0.05
k = 5  # negativos
# Vetores: centro (V) e contexto (U)
V_c = (rng.standard_normal((V, d)) / np.sqrt(d))
U_o = (rng.standard_normal((V, d)) / np.sqrt(d))

def sigmoid(x): return 1/(1+np.exp(-x))

# --- 3) Treino ---
def train(epochs=5):
    global V_c, U_o
    for ep in range(epochs):
        rng.shuffle(pairs)
        loss = 0.0
        for c,o in pairs:
            vc = V_c[c]          # (d,)
            uo = U_o[o]          # (d,)
            negs = sample_negatives(k)
            # evitar amostrar o positivo como negativo (opcional)
            negs = np.array([w for w in negs if w!=o])
            if len(negs)<k: 
                extra = sample_negatives(k-len(negs))
                negs = np.concatenate([negs, extra])

            # positivo
            score_pos = uo @ vc
            sig_pos = sigmoid(score_pos)
            loss += -np.log(sig_pos + 1e-10)

            # negativos
            u_negs = U_o[negs]            # (k,d)
            scores_neg = u_negs @ vc      # (k,)
            sig_neg = sigmoid(-scores_neg)
            loss += -np.sum(np.log(sig_neg + 1e-10))

            # grad vc
            grad_v = (sig_pos - 1.0)*uo + np.sum(sigmoid(scores_neg)[:,None]*u_negs, axis=0)
            # grad uo e u_negs
            grad_uo = (sig_pos - 1.0)*vc
            grad_unegs = (sigmoid(scores_neg)[:,None])*vc[None,:]

            # atualizações
            V_c[c] -= lr * grad_v
            U_o[o] -= lr * grad_uo
            U_o[negs] -= lr * grad_unegs

        print(f"Época {ep+1}: loss={loss/len(pairs):.4f}")

train(epochs=8)

# --- 4) Similaridades
def most_similar(word, topn=5):
    wid = word2id[word]
    # vetor final: média de V e U (prática comum)
    W = (V_c + U_o)/2
    w = W[wid]
    sims = W @ w / (np.linalg.norm(W, axis=1)*np.linalg.norm(w)+1e-9)
    best = np.argsort(-sims)
    return [(id2word[i], float(sims[i])) for i in best if i!=wid][:topn]

print("Mais similares a 'brasil':", most_similar("brasil"))
print("Mais similares a 'casa':", most_similar("casa"))
print("Mais similares a 'pesquisa':", most_similar("pesquisa"))

Vocabulário: 373 Pares: 2546
Época 1: loss=4.0524
Época 2: loss=3.6783
Época 3: loss=3.2893
Época 4: loss=2.9787
Época 5: loss=2.7250
Época 6: loss=2.4952
Época 7: loss=2.3210
Época 8: loss=2.1815
Mais similares a 'brasil': [('cafe', 0.7007219158266396), ('produz', 0.6932317867634673), ('do', 0.6689019618394818), ('exporta', 0.6194163760682415), ('noticias', 0.578653985473427)]
Mais similares a 'casa': [('fora', 0.607225408197346), ('cursos', 0.5778117056747542), ('entre', 0.5147343566593109), ('de', 0.4388081772317695), ('clinica', 0.4321712309960902)]
Mais similares a 'pesquisa': [('organiza', 0.5906688431001758), ('seminario', 0.5799002315004447), ('dados', 0.5586281700612934), ('em', 0.5477635374483255), ('artificial', 0.52616264811349)]


### CBOW

No **CBOW**, agregamos os vetores de contexto (p.ex., soma ou média) para prever a central:
$$
\mathbf{h} = \sum_{j\in \mathcal{C}(t)} \mathbf{u}_{w_{t+j}},
\qquad
P(w_t\mid \mathcal{C}(t)) = \text{softmax}(V\,\mathbf{h})
$$
Os gradientes seguem de forma análoga ao caso Skip-gram, com $\mathbf{h}$ no lugar de $\mathbf{v}_c$.

**Observações práticas**:
- **Skip-gram** tende a funcionar melhor para palavras raras (mais pares por central).
- **CBOW** é mais rápido e liso, pois agrega contexto (menos ruído).
- Janela pequena → mais **sintaxe**; janela grande → mais **semântica**.

## 3) GloVe — Global Vectors (Pennington, Socher, Manning, 2014)

Constrói matriz de **coocorrência** $X \in \mathbb{R}^{|V|\times |V|}$, onde $X_{ij}$ é a contagem (ponderada por distância) de $j$ no contexto de $i$.

**Objetivo:**
$$
J = \sum_{i,j} f(X_{ij})\left( \mathbf{w}_i^\top \tilde{\mathbf{w}}_j + b_i + \tilde{b}_j - \log X_{ij} \right)^2
$$
com
$$
f(x)=\begin{cases}
(x/x_{\max})^\alpha, & \text{se } x < x_{\max}\\
1, & \text{caso contrário}
\end{cases}
\quad\text{(tipicamente } x_{\max}=100,\ \alpha=\tfrac{3}{4}\text{)}
$$

**Gradientes:**
Defina o erro
$
E_{ij} = \mathbf{w}_i^\top \tilde{\mathbf{w}}_j + b_i + \tilde{b}_j - \log X_{ij}.
$
Então
$$
\frac{\partial J}{\partial \mathbf{w}_i} = \sum_j f(X_{ij})\,E_{ij}\,\tilde{\mathbf{w}}_j,\quad
\frac{\partial J}{\partial \tilde{\mathbf{w}}_j} = \sum_i f(X_{ij})\,E_{ij}\,\mathbf{w}_i
$$
$$
\frac{\partial J}{\partial b_i} = \sum_j f(X_{ij})\,E_{ij},\quad
\frac{\partial J}{\partial \tilde{b}_j} = \sum_i f(X_{ij})\,E_{ij}
$$

### Intuição da função objetivo do GloVe

**Objetivo:**
$$
J = \sum_{i,j} f(X_{ij})\left( \mathbf{w}_i^\top \tilde{\mathbf{w}}_j + b_i + \tilde{b}_j - \log X_{ij} \right)^2
$$

- $X_{ij}$ = número de vezes que a palavra $j$ aparece no contexto de $i$ (coocorrência).  
- $\mathbf{w}_i, \tilde{\mathbf{w}}_j$ = vetores da palavra $i$ e do contexto $j$.  
- $b_i, \tilde{b}_j$ = vieses (ajustes de escala).  
- $f(X_{ij})$ = função de ponderação que controla o peso de cada par $(i,j)$.  

---

#### O que essa equação está dizendo?

1. **Queremos aproximar:**
$$
\mathbf{w}_i^\top \tilde{\mathbf{w}}_j + b_i + \tilde{b}_j \;\;\approx\;\; \log X_{ij}
$$

Ou seja:  
o **produto interno dos vetores** (mais biases) deve se alinhar com o **log da coocorrência**.  

- Se duas palavras aparecem muito juntas → $\log X_{ij}$ alto → vetor $\mathbf{w}_i$ precisa ficar próximo de $\tilde{\mathbf{w}}_j$.  
- Se raramente aparecem juntas → $\log X_{ij}$ baixo → o produto interno deve ser pequeno ou negativo.

---

2. **Por que usar $\log X_{ij}$ em vez de $X_{ij}$?**  
- As contagens brutas crescem muito rápido.  
- O log “comprime” grandes diferenças, permitindo comparar palavras comuns e raras de forma mais equilibrada.

---

3. **Por que a função de ponderação $f(x)$?**  
$$
f(x)=\begin{cases}
(x/x_{\max})^\alpha & \text{se } x < x_{\max} \\
1 & \text{caso contrário}
\end{cases}
$$

- Pares muito frequentes (ex.: “de”, “o”) **não devem dominar** o treinamento → $f(x)$ os limita.  
- Pares muito raros (coocorrência próxima de 0) **não são confiáveis** → $f(x)$ os enfraquece.  
- Tipicamente: $x_{\max}=100$, $\alpha=3/4$.

---

#### E os gradientes?

Definindo o erro:
$$
E_{ij} = \mathbf{w}_i^\top \tilde{\mathbf{w}}_j + b_i + \tilde{b}_j - \log X_{ij}
$$

- **Para $\mathbf{w}_i$:**  
  move o vetor da palavra $i$ para mais perto ou mais longe de $\tilde{\mathbf{w}}_j$, dependendo do erro.  
- **Para $\tilde{\mathbf{w}}_j$:**  
  ajuste simétrico para o vetor de contexto.  
- **Para os bias $b_i, \tilde{b}_j$:**  
  pequenos deslocamentos que ajudam a calibrar os valores sem precisar alterar todo o vetor.

---

### Intuição global do GloVe

- Cada par de palavras é uma **equação de regressão**:  
  “o quanto $i$ e $j$ coocorrem deve ser igual ao produto dos seus vetores”.  
- O treinamento ajusta todos os vetores para que essas equações sejam satisfeitas “o melhor possível”.  
- O resultado são embeddings que preservam **relações semânticas lineares** (ex.: rei − homem + mulher ≈ rainha).

---

In [24]:
# ===== Mini-GloVe 
import numpy as np
from collections import defaultdict
rng = np.random.default_rng(0)

# 1) Matriz de coocorrência esparsa X com janela simétrica ponderada (1/dist)
window = 5  # janela um pouco maior costuma ajudar semântica
X = defaultdict(float)
for s in sentences:
    ids_s = [word2id[w] for w in s if w in word2id]
    for t, wi in enumerate(ids_s):
        L = max(0, t-window); R = min(len(ids_s), t+window+1)
        for j in range(L, R):
            if j == t: 
                continue
            wj = ids_s[j]
            dist = abs(j - t)
            X[(wi, wj)] += 1.0 / dist

V = len(word2id)
d = 100                 # um pouco maior que no toy anterior
W  = rng.normal(scale=0.1, size=(V, d))
Wt = rng.normal(scale=0.1, size=(V, d))
b  = np.zeros(V)
bt = np.zeros(V)

xmax, alpha = 100.0, 0.75
def f(x):
    return (x/xmax)**alpha if x < xmax else 1.0

# AdaGrad
eps = 1e-8
gW  = np.zeros_like(W)
gWt = np.zeros_like(Wt)
gb  = np.zeros_like(b)
gbt = np.zeros_like(bt)

keys = list(X.keys())

def glove_train(iters=70, lr=0.05):
    for it in range(iters):
        rng.shuffle(keys)
        J = 0.0
        for (i, j) in keys:
            xij = X[(i, j)]
            wij = W[i]; wjt = Wt[j]
            err = wij @ wjt + b[i] + bt[j] - np.log(xij)
            fij = f(xij)
            J += 0.5 * fij * (err**2)

            # gradientes
            grad_wi = fij * err * wjt
            grad_wj = fij * err * wij
            grad_bi = fij * err
            grad_bj = fij * err

            # AdaGrad updates
            gW[i]  += grad_wi**2
            gWt[j] += grad_wj**2
            gb[i]  += grad_bi**2
            gbt[j] += grad_bj**2

            W[i]  -= lr * grad_wi / (np.sqrt(gW[i])  + eps)
            Wt[j] -= lr * grad_wj / (np.sqrt(gWt[j]) + eps)
            b[i]  -= lr * grad_bi / (np.sqrt(gb[i])  + eps)
            bt[j] -= lr * grad_bj / (np.sqrt(gbt[j]) + eps)

        if (it+1) % 10 == 0 or it < 5:
            print(f"[GloVe] it {it+1:02d} | J/|X| = {J/len(keys):.4f}")

glove_train(iters=60, lr=0.05)

# Vetores finais práticos: soma
E_glove = W + Wt

def most_similar_glove(word, topn=10):
    if word not in word2id:
        raise KeyError(f"'{word}' não está no vocabulário do GloVe.")
    wid = word2id[word]
    w = E_glove[wid]
    norms = np.linalg.norm(E_glove, axis=1) * (np.linalg.norm(w) + 1e-9)
    sims = (E_glove @ w) / (norms + 1e-9)
    order = np.argsort(-sims)
    return [(id2word[i], float(sims[i])) for i in order if i != wid][:topn]

print("\n[GloVe] vizinhos de 'rio':", most_similar_glove("rio")[:7])
print("[GloVe] vizinhos de 'futebol':", most_similar_glove("futebol")[:7])
print("[GloVe] vizinhos de 'universidade':", most_similar_glove("universidade")[:7])

[GloVe] it 01 | J/|X| = 0.0076
[GloVe] it 02 | J/|X| = 0.0022
[GloVe] it 03 | J/|X| = 0.0007
[GloVe] it 04 | J/|X| = 0.0003
[GloVe] it 05 | J/|X| = 0.0002
[GloVe] it 10 | J/|X| = 0.0000
[GloVe] it 20 | J/|X| = 0.0000
[GloVe] it 30 | J/|X| = 0.0000
[GloVe] it 40 | J/|X| = 0.0000
[GloVe] it 50 | J/|X| = 0.0000
[GloVe] it 60 | J/|X| = 0.0000

[GloVe] vizinhos de 'rio': [('o', 0.5710707482288602), ('no', 0.5353837859189712), ('brasil', 0.35991923669257053), ('classico', 0.3367246458955786), ('jogo', 0.3336451727563668), ('de', 0.29578503725832717), ('estadio', 0.272847955258618)]
[GloVe] vizinhos de 'futebol': [('de', 0.4755553378757443), ('laboratorio', 0.40369747393709793), ('entre', 0.34434275379207097), ('sala', 0.33290145334383386), ('doenca', 0.330268647918877), ('atendimento', 0.31925752517141215), ('com', 0.31764987376007975)]
[GloVe] vizinhos de 'universidade': [('a', 0.6196808997322713), ('e', 0.4728297843586192), ('tem', 0.455252503383725), ('praia', 0.4310534321557508), ('finai

## 4) FastText (Bojanowski et al., 2017)

O **FastText** foi proposto pelo Facebook em 2017 como uma extensão natural do Word2Vec.  
A grande diferença é que **cada palavra não é tratada como indivisível**, mas como uma soma de vetores de **sub-palavras** (n-gramas de caracteres).

---

### Ideia central
- Para cada palavra $w$, consideramos todos os seus **n-gramas de caracteres** (tipicamente $n \in [3,6]$).  
- Adicionam-se símbolos de início/fim (`<`, `>`) para capturar prefixos e sufixos.  

**Exemplo (n=3):**
- Palavra: `"casa"`  
- n-gramas: `<ca`, `cas`, `asa`, `sa>`  

Cada n-grama tem um vetor $\mathbf{z}_g$.  
O vetor final da palavra é a soma (ou média) desses vetores:

$$
\mathbf{z}(w)=\sum_{g\in G_w} \mathbf{z}_g
$$

---

### Como o treinamento funciona?
O modelo FastText treina quase da mesma forma que o Skip-gram com *negative sampling*:  

- Antes: no Skip-gram padrão, usamos o vetor da palavra central $\mathbf{v}_c$.  
- Agora: substituímos $\mathbf{v}_c$ por $\mathbf{z}(w_c)$, que é construído a partir de n-gramas.  

Assim, **as mesmas equações de probabilidade e gradientes** do Word2Vec continuam válidas, mas agora a informação é **compartilhada entre palavras com pedaços semelhantes**.

---

### Por que isso é importante?

1. **Palavras raras**  
   - Word2Vec e GloVe precisam ver muitas ocorrências para aprender um bom vetor.  
   - FastText generaliza porque compartilha n-gramas entre palavras.  
   - Ex.: "corrida", "correr", "corredor" → compartilham `corr`.

2. **Palavras fora do vocabulário (OOV)**  
   - Em Word2Vec/GloVe, se a palavra não está no vocabulário → não existe vetor.  
   - Em FastText, ainda podemos gerar um embedding **somando os n-gramas** (mesmo que a palavra nunca tenha aparecido).  
   - Ex.: "cachorrinho" → pode ser decomposta em sub-palavras conhecidas de "cachorro".

3. **Morfologia rica (como no português)**  
   - Idiomas flexivos têm muitas variações de uma mesma raiz (plural, feminino, conjugação verbal).  
   - FastText é capaz de capturar essas regularidades sem precisar de milhões de ocorrências por forma.

---

### Comparação rápida
- **Word2Vec/GloVe:** vetor único por palavra (não entende morfologia).  
- **FastText:** vetor = soma de sub-palavras → mais robusto para línguas com flexão.  
- **Limitação:** continua sendo um modelo **estático** (não resolve polissemia como BERT/ELMo).

---

### Vantagens práticas
- Funciona muito bem em **português** e outras línguas ricas em morfologia.  
- Permite embeddings para **neologismos** e **palavras OOV**.  
- Modelo rápido, aproveitando a mesma mecânica do Skip-gram com Negative Sampling.  

In [25]:
from gensim.models import FastText

ft_model = FastText(
    sentences=sentences,
    vector_size=100,
    window=5,
    min_count=1,
    workers=2,   
    sg=1       
)

def most_similar_fasttext(word, topn=10):
    return ft_model.wv.most_similar(word, topn=topn)

print("[FastText] vizinhos de 'praia':", most_similar_fasttext("praia")[:7])
print("[FastText] vizinhos de 'hospital':", most_similar_fasttext("hospital")[:7])

oov = "futebolzinho"
vec_oov = ft_model.wv[oov]  
print("[FastText] vizinhos de 'futebol' pelo FastText:", most_similar_fasttext("futebol")[:7])

[FastText] vizinhos de 'praia': [('praias', 0.6487528681755066), ('cafeteria', 0.5528571009635925), ('cafeterias', 0.526585578918457), ('amazonia', 0.4928141236305237), ('cafe', 0.49207791686058044), ('corta', 0.4876181483268738), ('estadio', 0.4657224118709564)]
[FastText] vizinhos de 'hospital': [('grama', 0.5155500173568726), ('capital', 0.5016581416130066), ('apita', 0.47575175762176514), ('dados', 0.47549065947532654), ('treinador', 0.4560692608356476), ('cafe', 0.4556812644004822), ('gramado', 0.4540213346481323)]
[FastText] vizinhos de 'futebol' pelo FastText: [('pacientes', 0.444522500038147), ('seminario', 0.4211379587650299), ('aplica', 0.4197556674480438), ('dados', 0.4185933768749237), ('comunidade', 0.39609429240226746), ('de', 0.39179563522338867), ('comunidades', 0.38463878631591797)]


In [26]:
W_w2v = (V_c + U_o) / 2

def vec_w2v(token):
    if token not in word2id:
        raise KeyError(f"'{token}' não está no vocabulário do modelo W2V caseiro.")
    return W_w2v[word2id[token]]

def vec_glove(token):
    if token not in word2id:
        raise KeyError(f"'{token}' não está no vocabulário do GloVe.")
    return E_glove[word2id[token]]

def vec_fasttext(token):
    return ft_model.wv[token]  # lida com OOV via subword

def most_similar_generic(model_name, token, topn=10):
    if model_name == "w2v":
        w = vec_w2v(token)
        E = W_w2v
    elif model_name == "glove":
        w = vec_glove(token)
        E = E_glove
    elif model_name == "fasttext":
        return ft_model.wv.most_similar(token, topn=topn)
    else:
        raise ValueError("model_name deve ser 'w2v', 'glove' ou 'fasttext'.")
    sims = E @ w / (np.linalg.norm(E, axis=1) * (np.linalg.norm(w) + 1e-9) + 1e-9)
    order = np.argsort(-sims)
    wid = word2id.get(token, None)
    result = []
    for i in order:
        if wid is not None and i == wid:
            continue
        result.append((id2word[i], float(sims[i])))
        if len(result) >= topn:
            break
    return result

print("[Comparação] W2V vizinhos de 'rio':", most_similar_generic("w2v","rio")[:7])
print("[Comparação] GloVe vizinhos de 'rio':", most_similar_generic("glove","rio")[:7])
print("[Comparação] FastText vizinhos de 'rio':", most_similar_generic("fasttext","rio")[:7])

[Comparação] W2V vizinhos de 'rio': [('negro', 0.7376405481555982), ('encontra', 0.711974304423906), ('porto', 0.6401831007401755), ('universitario', 0.5904406810028948), ('janeiro', 0.5602900814670095), ('nada', 0.46245816998878786), ('no', 0.442251491364128)]
[Comparação] GloVe vizinhos de 'rio': [('o', 0.5710707482288602), ('no', 0.5353837859189712), ('brasil', 0.35991923669257053), ('classico', 0.3367246458955786), ('jogo', 0.3336451727563668), ('de', 0.29578503725832717), ('estadio', 0.272847955258618)]
[Comparação] FastText vizinhos de 'rio': [('laboratorio', 0.5692906975746155), ('cafeterias', 0.5472309589385986), ('prontuario', 0.5312071442604065), ('frio', 0.5243507027626038), ('dados', 0.4971526563167572), ('rios', 0.48958051204681396), ('estadio', 0.48705339431762695)]


In [31]:
import numpy as np
from gensim.models import Word2Vec, FastText
from gensim.models.phrases import Phrases, Phraser

# 1) Tokenização e bigramas
sentences = [s.split() for s in corpus]

# aprende bigramas no seu corpus
phrases = Phrases(sentences, min_count=2, threshold=5, delimiter='_')
bigram = Phraser(phrases)
sentences_bg = [bigram[s] for s in sentences]

print("Exemplo com bigramas:", sentences_bg[0][:15])

vector_size = 100
window = 5
min_count = 1
workers = 2

# Word2Vec CBOW
w2v_cbow = Word2Vec(
    sentences=sentences_bg,
    vector_size=vector_size,
    window=window,
    min_count=min_count,
    workers=workers,
    sg=0,   # CBOW
    negative=10,
    epochs=50
)

# Word2Vec Skip-gram
w2v_sg = Word2Vec(
    sentences=sentences_bg,
    vector_size=vector_size,
    window=window,
    min_count=min_count,
    workers=workers,
    sg=1,   # Skip-gram
    negative=10,
    epochs=50
)

# FastText Skip-gram (subwords)
ft_sg = FastText(
    sentences=sentences_bg,
    vector_size=vector_size,
    window=window,
    min_count=min_count,
    workers=workers,
    sg=1,   # Skip-gram
    negative=10,
    epochs=50
)

def most_similar(model, token, topn=7):
    try:
        return model.wv.most_similar(token, topn=topn)
    except KeyError as e:
        return [("<<OOV ou ausente no vocabulário>>", 0.0)]

def phrase_mean_vec(model, phrase_tokens):
    vecs = []
    for t in phrase_tokens:
        if t in model.wv:
            vecs.append(model.wv[t])
    if not vecs:
        raise KeyError(f"Nenhum token com vetor em {phrase_tokens}")
    return np.mean(vecs, axis=0)

def analogy(model, a, b, c, topn=7):
    # a : b :: c : ?
    def get_vec(term):
        if term in model.wv:
            return model.wv[term]
        else:
            toks = term.split('_')
            return phrase_mean_vec(model, toks)

    va, vb, vc = get_vec(a), get_vec(b), get_vec(c)
    target = vb - va + vc
    sims = model.wv.cosine_similarities(target, model.wv.vectors)
    order = np.argsort(-sims)
    idx2tok = model.wv.index_to_key
    # excluir termos da consulta
    excl = {a, b, c}
    ans = []
    for idx in order:
        tok = idx2tok[idx]
        if tok in excl: 
            continue
        ans.append((tok, float(sims[idx])))
        if len(ans) >= topn:
            break
    return ans

probes = ["rio", "rio_de_janeiro", "sao_paulo", "futebol", "universidade", "hospital", "amazonia", "cafe", "maracana"]

print("\n=== Vizinhos mais similares (CBOW) ===")
for p in probes:
    print(f"{p:>15} -> {most_similar(w2v_cbow, p)[:5]}")

print("\n=== Vizinhos mais similares (Skip-gram) ===")
for p in probes:
    print(f"{p:>15} -> {most_similar(w2v_sg, p)[:5]}")

print("\n=== Vizinhos mais similares (FastText) ===")
for p in probes:
    print(f"{p:>15} -> {most_similar(ft_sg, p)[:5]}")

print("\n=== Analogia: rio_de_janeiro : maracana :: sao_paulo : ? ===")
print("CBOW     ->", analogy(w2v_cbow, "rio_de_janeiro", "maracana", "sao_paulo", topn=5))
print("Skip-gram->", analogy(w2v_sg,   "rio_de_janeiro", "maracana", "sao_paulo", topn=5))
print("FastText ->", analogy(ft_sg,    "rio_de_janeiro", "maracana", "sao_paulo", topn=5))

oov = "futebolzinho"
vec_oov = ft_sg.wv[oov] 
print(f"\nOOV '{oov}': norma do vetor FastText =", float(np.linalg.norm(vec_oov)))
print("Vizinhos de 'futebol' no FastText ->", most_similar(ft_sg, "futebol")[:7])

Exemplo com bigramas: ['o', 'brasil', 'gosta', 'de_futebol']

=== Vizinhos mais similares (CBOW) ===
            rio -> [('o', 0.9995489716529846), ('no', 0.9995225667953491), ('e', 0.99946528673172), ('a', 0.9994440674781799), ('do', 0.9994298815727234)]
 rio_de_janeiro -> [('<<OOV ou ausente no vocabulário>>', 0.0)]
      sao_paulo -> [('a', 0.9990635514259338), ('no', 0.999021053314209), ('o', 0.9990162253379822), ('de', 0.9989767074584961), ('tem', 0.9989765882492065)]
        futebol -> [('do', 0.9990296363830566), ('a', 0.9989714026451111), ('no', 0.9989632368087769), ('de', 0.9989615082740784), ('organiza', 0.998950719833374)]
   universidade -> [('na', 0.9993563294410706), ('a', 0.9993546009063721), ('o', 0.9993136525154114), ('tem', 0.999302089214325), ('e', 0.9992493391036987)]
       hospital -> [('e', 0.9994431734085083), ('a', 0.9994238615036011), ('no', 0.9993908405303955), ('o', 0.9993587136268616), ('por', 0.9993432760238647)]
       amazonia -> [('o', 0.999463677406311

## 5) Avaliação de embeddings

**Intrínseca**
- **Similaridade**: correlação com anotações humanas (WordSim, RG65 etc.).
- **Analogia**: avaliar se $\mathbf{w}_{\text{king}} - \mathbf{w}_{\text{man}} + \mathbf{w}_{\text{woman}} \approx \mathbf{w}_{\text{queen}}$.
- **Visualização**: TSNE/UMAP de vizinhanças semânticas.

**Extrínseca**
- Plug-and-play em tarefas (NER, POS, classificação) e medir F1/acc.

> Em PT-BR, tentem analogias como “Brasil : Brasília :: França : ?”.

In [27]:
import re
def tokenize_phrase(text):
    # separa por espaço, mantém tokens tal como no corpus (sem acento/caixa já condizentes)
    return [t for t in re.split(r"\s+", text.strip()) if t]

def phrase_vec(model, phrase):
    toks = tokenize_phrase(phrase)
    vecs = []
    for t in toks:
        try:
            if model == "w2v":
                vecs.append(vec_w2v(t))
            elif model == "glove":
                vecs.append(vec_glove(t))
            elif model == "fasttext":
                vecs.append(vec_fasttext(t))
            else:
                raise ValueError
        except KeyError:
            # ignora tokens OOV no W2V/GloVe; FastText deve cobrir quase tudo
            pass
    if not vecs:
        raise KeyError(f"Nenhum token com vetor em '{phrase}'.")
    return np.mean(vecs, axis=0)

def analogy(a, b, c, model="w2v", topn=10):
    # Resolve analogia: a : b :: c : ?
    va = phrase_vec(model, a)
    vb = phrase_vec(model, b)
    vc = phrase_vec(model, c)
    target = vb - va + vc

    # matriz de referência
    if model == "w2v":
        E = W_w2v
        idx2tok = id2word
    elif model == "glove":
        E = E_glove
        idx2tok = id2word
    elif model == "fasttext":
        # Para FastText, criamos uma matriz E para termos do vocabulário conhecido
        vocab_ft = list(ft_model.wv.key_to_index.keys())
        E = np.vstack([ft_model.wv[w] for w in vocab_ft])
        idx2tok = {i:w for i,w in enumerate(vocab_ft)}
    else:
        raise ValueError

    sims = E @ target / (np.linalg.norm(E, axis=1) * (np.linalg.norm(target) + 1e-9) + 1e-9)
    order = np.argsort(-sims)

    # Remove termos presentes na entrada (quando possível mapear para índice)
    exclude = set()
    for phr in [a,b,c]:
        for t in tokenize_phrase(phr):
            # mapear para índice correspondente
            if model in ("w2v","glove"):
                if t in word2id:
                    exclude.add(word2id[t])
            else:
                # FastText: mapeamento por string
                pass

    ans = []
    for i in order:
        tok = idx2tok[i]
        # excluir entradas
        if model in ("w2v","glove"):
            if i in exclude:
                continue
        if tok in set(tokenize_phrase(a) + tokenize_phrase(b) + tokenize_phrase(c)):
            continue
        ans.append((tok, float(sims[i])))
        if len(ans) >= topn:
            break
    return ans

# Exemplos: cidades e estádios (tokens disponíveis no corpus)
print("\n[Analogy W2V] 'rio' : 'maracana' :: 'sao paulo' : ?", analogy("rio","maracana","sao paulo","w2v",topn=5))
print("[Analogy GloVe] 'rio' : 'maracana' :: 'sao paulo' : ?", analogy("rio","maracana","sao paulo","glove",topn=5))
print("[Analogy FT] 'rio de janeiro' : 'maracana' :: 'sao paulo' : ?", analogy("rio de janeiro","maracana","sao paulo","fasttext",topn=5))


[Analogy W2V] 'rio' : 'maracana' :: 'sao paulo' : ? [('referencia', 0.5935763774089288), ('usa', 0.5147199488882565), ('e', 0.5080159046442877), ('em', 0.43705912410461695), ('trilhas', 0.4295243267130231)]
[Analogy GloVe] 'rio' : 'maracana' :: 'sao paulo' : ? [('manaus', 0.5674081380262637), ('cor', 0.5178318935988463), ('ipanema', 0.39842362740131737), ('bonita', 0.3836451174195685), ('startups', 0.36872891980181605)]
[Analogy FT] 'rio de janeiro' : 'maracana' :: 'sao paulo' : ? [('pao', 0.32736703753471375), ('ameaca', 0.30053210258483887), ('atendimento', 0.2784554362297058), ('learning', 0.27785807847976685), ('parque', 0.274782657623291)]


## Referências

- Mikolov, T., Chen, K., Corrado, G., & Dean, J. (2013). Efficient Estimation of Word Representations in Vector Space. arXiv:1301.3781.
- Pennington, J., Socher, R., & Manning, C. (2014). GloVe: Global Vectors for Word Representation. In *EMNLP 2014*, 1532–1543.
- Bojanowski, P., Grave, E., Joulin, A., & Mikolov, T. (2017). Enriching Word Vectors with Subword Information. *TACL*, 5, 135–146.
- Firth, J. R. (1957). *A Synopsis of Linguistic Theory 1930–1955*. Oxford.
- Levy, O., Goldberg, Y., & Dagan, I. (2015). Improving Distributional Similarity with Lessons Learned from Word Embeddings. *TACL*, 3, 211–225.